In [1]:
import re
import sys
import string
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding

Using Theano backend.
Using gpu device 0: GeForce GT 755M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)


In [2]:
rawtext = open('wonderland.txt','r').read().split('\n')
rawtext = ' '.join(rawtext)
rawtext = [word.strip(string.punctuation) for word in rawtext.split()]
rawtext = ' '.join(rawtext)
rawtext = rawtext.replace('-', ' ')
rawtext = ' '.join(rawtext.split())

In [3]:
all_words = rawtext.split()
unique_words = sorted(list(set(all_words)))
n_vocab = len(unique_words)
print "Total Vocab:", n_vocab
word_to_int = dict((w, i) for i, w in enumerate(unique_words))
int_to_word = dict((i, w) for i, w in enumerate(unique_words))

Total Vocab: 3063


In [4]:
raw_text = rawtext.split()
n_words = len(raw_text)
print n_words

26694


In [5]:
seq_length = 100
dataX = []
dataY = []
for i in xrange(0, n_words - seq_length):
    seq_in  = raw_text[i: i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print 'Total patterns:', n_patterns

Total patterns: 26594


In [6]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 256)           264192      lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 256)           0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 3063L)         787191      dropout_1[0][0]                  
Total params: 1051383
____________________________________________________________________________________________________
None


In [8]:
# define the checkpoint
filepath="data/weights/word-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [74]:
model.fit(X_train, Y_train, nb_epoch=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
26594/26594 [==============================] - 143s - loss: 6.3193   
Epoch 2/10
26594/26594 [==============================] - 143s - loss: 6.2409   
Epoch 3/10
26594/26594 [==============================] - 143s - loss: 6.2079   
Epoch 4/10
26594/26594 [==============================] - 142s - loss: 6.1717   
Epoch 5/10
26594/26594 [==============================] - 141s - loss: 6.1340   
Epoch 6/10
26594/26594 [==============================] - 142s - loss: 6.0868   
Epoch 7/10
26594/26594 [==============================] - 144s - loss: 6.0397   
Epoch 8/10
26594/26594 [==============================] - 142s - loss: 5.9845   
Epoch 9/10
26594/26594 [==============================] - 143s - loss: 5.9268   
Epoch 10/10
26594/26594 [==============================] - 142s - loss: 5.8583   


In [9]:
# load the network weights
filename = "data/weights/weights-improvement-18-2.8349.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
model.fit(X_train, Y_train, nb_epoch=80, batch_size=32, callbacks=callbacks_list)

Epoch 1/80
26594/26594 [==============================] - 213s - loss: 2.9220   
Epoch 2/80
26594/26594 [==============================] - 213s - loss: 2.8964   
Epoch 3/80
26594/26594 [==============================] - 213s - loss: 4.0208   
Epoch 4/80
26594/26594 [==============================] - 213s - loss: 6.3712   
Epoch 5/80
26594/26594 [==============================] - 213s - loss: 5.8421   
Epoch 6/80
26594/26594 [==============================] - 213s - loss: 5.4543   
Epoch 7/80
26594/26594 [==============================] - 214s - loss: 5.2228   
Epoch 8/80
26594/26594 [==============================] - 214s - loss: 4.5284   
Epoch 9/80
26594/26594 [==============================] - 212s - loss: 4.5802   
Epoch 10/80
26594/26594 [==============================] - 212s - loss: 3.9312   
Epoch 11/80
26594/26594 [==============================] - 213s - loss: 3.6743   
Epoch 12/80
26594/26594 [==============================] - 213s - loss: 3.5348   
Epoch 13/80
26594/26594 [

In [89]:
# load the network weights
filename = "weights-improvement-18-2.8349.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
result = []
print "Seed:"
print "\"", ' '.join([int_to_word[value] for value in pattern]), "\""
for i in xrange(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    result.append(int_to_word[index])
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nGenerated Sequence:"
print ' '.join(result)
print "\nDone."  

Seed:
" he had to ask his neighbour to tell him A nice muddle their slates'll be in before the trial's over thought Alice One of the jurors had a pencil that squeaked This of course Alice could not stand and she went round the court and got behind him and very soon found an opportunity of taking it away She did it so quickly that the poor little juror it was Bill the Lizard could not make out at all what had become of it so after hunting all about for it he was obliged to write with one finger for "

Generated Sequence:
the rest the the the and it that of the the and talking I she I the understand to the the the happen key key the spell Look and she So shan't the would of at this Footman's upon I she to to the that to pebbles the the the the dance Suppose you Alice's it it May who much of thought Alice in a perhaps half and so to to to to pebbles the the it the dance Will you won't you will you won't won't you you join join dance Thank I I think THAT said Duchess said the Alice said tr